In [1]:
from collections import Counter
import numpy as np
import tskit


In [ ]:
# Generate these trees by running pytest tests/test_inference.py
ts_file = "../tests/data/cache/2020-02-13.ts"
ts = tskit.load(ts_file)
ts


In [ ]:
mut_labels = {}
mut_pos = []
mut_types = []

for mut in ts.mutations():
    site = ts.site(mut.site)
    pos = int(site.position)
    older_mut = mut.parent >= 0
    prev = ts.mutation(mut.parent).derived_state if older_mut else site.ancestral_state
    mut_labels[mut.id] = f"{prev}{pos}{mut.derived_state}"
    mut_pos.append(pos)
    mut_types.append(f"{prev}{mut.derived_state}")

# What is A1547-? One-base deletion in ORF1a.
Counter(mut_types)


In [ ]:
# There is one site having two mutations.
mut_pos_counts = Counter(mut_pos)
for k, v in mut_pos_counts.items():
    if v > 1:
        print(f"pos: {k}; count: {v}")


In [ ]:
# There should be no recurrent mutations this early.
all(np.array(list(Counter(mut_labels.values()).values()), dtype=int) == 1)


In [ ]:
# The majority of samples should be from the lineage B, with some from the lineage A.
node_labels = {}
pango_labels = []
for node in ts.nodes():
    if node.is_sample() and "Viridian_pangolin" in node.metadata:
        strain = node.metadata["strain"]
        pango = node.metadata["Viridian_pangolin"]
        node_labels[node.id] = f"{pango}\n{strain}"
        pango_labels.append(pango)
    else:
        node_labels[node.id] = f"{node.id}"
Counter(pango_labels)


In [ ]:
# Check polytomy at the top.
ts.first().num_children_array


In [ ]:
# Note that lineages A and B are characterised by differences at two site positions: 8782 and 28144.
# Lineage B samples have 8782C and 28144T, whereas lineage A samples have 8782T and 28144C.
# 
# The reference Wuhan-Hu-1 belongs to lineage B, and is arbitrarily chosen as the root here.
# So, we should expect to see samples from lineage A grouped by two mutations: C8782T and T28144C.
label_style = (
    ".mut > .lab {font-size: 80%; transform: rotate(-25deg) translate(0px)}"
    ".node > .lab {font-size: 80%}"
    ".leaf > .lab {text-anchor: middle; transform: rotate(-35deg) translateY(10px)}"
)

ts.draw_svg(
    node_labels=node_labels,
    mutation_labels=mut_labels,
    y_axis=True,
    time_scale="rank",
    x_scale="treewise",
    size=(950, 1500),
    style=label_style,
)


In [ ]:
# Some of these nodes are unary nodes shown above.
# That is, nodes 9, 45, 54.
for node in ts.nodes():
    if "date_added" in node.metadata:
        print(node)
